# Hill Climbing

In [1]:
import random
from collections import namedtuple

In [2]:
random.seed(42)

## Cookbook

### List comprehentions

In [3]:
[x for x in range(3)]

[0, 1, 2]

In [4]:
[
    (x, y)
    for x in range(3)
    for y in range(3)
]

[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]

### list#remove

In [5]:
l = 'computer science is no more about computers than astronomy is about telescopes'.split()
# __ by Edsger W. Dijkstra

In [6]:
l

['computer',
 'science',
 'is',
 'no',
 'more',
 'about',
 'computers',
 'than',
 'astronomy',
 'is',
 'about',
 'telescopes']

In [7]:
l.remove('is')

In [8]:
l

['computer',
 'science',
 'no',
 'more',
 'about',
 'computers',
 'than',
 'astronomy',
 'is',
 'about',
 'telescopes']

### random

In [9]:
random.seed(42) # run this to make your code reproducible

In [10]:
print(random.random()) # returns a random number in the interval [0, 1).
print(random.random())
print(random.random())
print(random.random())
print(random.random())

0.6394267984578837
0.025010755222666936
0.27502931836911926
0.22321073814882275
0.7364712141640124


In [11]:
print(random.random() < 0.3) # true 30% of times
print(random.random() < 0.3)
print(random.random() < 0.3)
print(random.random() < 0.3)
print(random.random() < 0.3)

False
False
True
False
True


In [12]:
print(random.choice(l)) # chooses a number from l at random
print(random.choice(l))
print(random.choice(l))
print(random.choice(l))
print(random.choice(l))

more
more
is
about
computer


In [13]:
print(random.sample(l, k=2)) # chooses k=2 numbers from l at random
print(random.sample(l, k=2))
print(random.sample(l, k=2))
print(random.sample(l, k=2))
print(random.sample(l, k=2))

['is', 'more']
['telescopes', 'is']
['than', 'more']
['astronomy', 'about']
['about', 'computer']


### _global_

In [14]:
x = 0
def f(): # You can read global variables
    print('read ', x)
f()

read  0


In [15]:
x = 0
def f(): # but you can only can not change them(why UnboundLocalError? I'll explain)
    print('read ', x)
    x = x + 1
f()
print(x)

UnboundLocalError: local variable 'x' referenced before assignment

In [16]:
x = 0
def f(): # but if you mark them as globals...
    global x
    print('read ', x)
    x = x + 1
f()
print(x)

read  0
1


In [17]:
x = 0
def f():
    global x
    x = x + 1
f()
print()

### Nested functions and _nonlocal_

In [18]:
def f(): # You can define functions withing functions(within function within functions...)
    def g():
        print('Hi!')
    g()
    g()
    g()

f()

Hi!
Hi!
Hi!


In [19]:
def f():
    message = 'HI!'
    def g(): # You can read messages from outer scope
        print(message)
    g()
    g()
    g()

f()

HI!
HI!
HI!


In [20]:
def f():
    message = 'HI!'
    def g(): # But if you assign them...
        print(message)
        message = 'hi'
    g()
    g()
    g()

f()

UnboundLocalError: local variable 'message' referenced before assignment

In [21]:
def f():
    message = 'HI!'
    def g(): # nonlocal to the rescue
        nonlocal message
        print(message)
        message = 'hi'
    g()
    g()
    g()

f()

HI!
hi
hi


## Knapsack Problem

![knapsack](https://upload.wikimedia.org/wikipedia/commons/thumb/f/fd/Knapsack.svg/250px-Knapsack.svg.png)

The knapsack problem or rucksack problem is a problem in combinatorial optimization: Given a set of items, each with a weight and a value, determine the number of each item to include in a collection so that the total weight is less than or equal to a given limit and the total value is as large as possible. It derives its name from the problem faced by someone who is constrained by a fixed-size knapsack and must fill it with the most valuable items.

[_From Wikipedia, the free encyclopedia_](From Wikipedia, the free encyclopedia)

### Inputs

* Input items
* Maximum weight

In [22]:
Item = namedtuple('Item', 'value weight')

In [23]:
Item(value=10, weight=2)

Item(value=10, weight=2)

In [24]:
items = [
    Item(
        value=max(0, random.gauss(5, 3)), 
        weight=max(0.1, random.gauss(10, 10))
    )
    for i in range(10000)
]

In [25]:
max_weight = 120

In [26]:
items[:10]

[Item(value=5.097870121296177, weight=4.110759788076699),
 Item(value=2.8598281099596248, weight=13.770646187017395),
 Item(value=6.101487786158066, weight=26.57937309512181),
 Item(value=7.34868310612216, weight=15.85588173554841),
 Item(value=3.2578371335610443, weight=17.112082423989964),
 Item(value=4.918528003349488, weight=12.968296489522924),
 Item(value=3.5008827432091025, weight=11.302291972593874),
 Item(value=6.073473923777699, weight=8.099521685906998),
 Item(value=3.868417476314377, weight=23.56056837947524),
 Item(value=7.137439081142394, weight=3.754498532159464)]

### Elements

In [27]:
desk = list(items)

knapsack = []
knapsack_weight = 0
knapsack_value = 0

In [28]:
# Moving from desk to knapsack
item = desk.pop()
knapsack.append(item)
knapsack_weight += item.weight
knapsack_value += item.value

In [29]:
# Moving from knapsack to desk
item = knapsack.pop()
desk.append(item)
knapsack_weight -= item.weight
knapsack_value -= item.value

### Hill Climbing: A single step

In our formulation an action is considered as removing an item _ki_ from the knapsack and moving item _di_ from the desk to the knapsack.

In [30]:
# Actions are of the form (ki, di, new_value)
actions = [
    (ki, di, knapsack_value + di.value - ki.value)
    for ki in knapsack
    for di in desk
    if knapsack_weight - ki.weight + di.weight < max_weight
]

In [31]:
# But we do not necessarily need to remove an item from the knapsack
actions.extend(
    (None, di, knapsack_value + di.value)
    for di in desk
    if knapsack_weight + di.weight < max_weight
)

In [32]:
# Finding the most valuable action
max_v = knapsack_value
max_ki = None
max_di = None
for ki, di, v in actions:
    if v > max_v:
        max_v = v
        max_ki = ki
        max_di = di

In [33]:
print(f'Value:\t{knapsack_value}')
print(f'Weight:\t{knapsack_weight}')

Value:	0.0
Weight:	0.0


In [34]:
# Performing the action
if max_di is not None:
    desk.remove(max_di)
    knapsack.append(max_di)
    knapsack_weight += max_di.weight
    knapsack_value += max_di.value
if max_ki is not None:
    knapsack.remove(max_ki)
    desk.append(max_ki)
    knapsack_weight -= max_ki.weight
    knapsack_value -= max_ki.value

In [35]:
print(f'Value:\t{knapsack_value}')
print(f'Weight:\t{knapsack_weight}')

Value:	16.58239338802688
Weight:	11.783649467897803


In [ ]:
# Greedy best-first
max_v = knapsack_value
max_ki = None
max_di = None
for ki, di, v in actions:
    if v > max_v:
        max_v = v
        max_ki = ki
        max_di = di
        break

In [ ]:
# Simulated annealing
max_v = knapsack_value
max_ki = None
max_di = None
for ki, di, v in actions:
    if (v > max_v) or (random.random() < 0.1):
        max_v = v
        max_ki = ki
        max_di = di
        break

### Multiple steps

In [36]:
def hill_climbing_step():
    global knapsack_weight, knapsack_value
    actions = [
        (ki, di, knapsack_value + di.value - ki.value)
        for ki in knapsack
        for di in desk
        if knapsack_weight - ki.weight + di.weight < max_weight
    ]
    actions.extend(
        (None, di, knapsack_value + di.value)
        for di in desk
        if knapsack_weight + di.weight < max_weight
    )

    max_v = knapsack_value
    max_ki = None
    max_di = None
    for ki, di, v in actions:
        if v > max_v:
            max_v = v
            max_ki = ki
            max_di = di

    if max_di is not None:
        desk.remove(max_di)
        knapsack.append(max_di)
        knapsack_weight += max_di.weight
        knapsack_value += max_di.value
    if max_ki is not None:
        knapsack.remove(max_ki)
        desk.append(max_ki)
        knapsack_weight -= max_ki.weight
        knapsack_value -= max_ki.value

In [37]:
desk = list(items)

knapsack = []
knapsack_weight = 0
knapsack_value = 0

print(f'Value:\t{knapsack_value}')
print(f'Weight:\t{knapsack_weight}')

Value:	0
Weight:	0


In [38]:
hill_climbing_step()
print(f'Value:\t{knapsack_value}')
print(f'Weight:\t{knapsack_weight}')

Value:	16.58239338802688
Weight:	11.783649467897803


### Complete Hill Climbing

In [39]:
def hill_climbing(items, max_weight, max_steps=100, best_first=False, anneal_rate=0):
    desk = list(items)
    
    knapsack = []
    knapsack_weight = 0
    knapsack_value = 0
    
    def hill_climbing_step():
        nonlocal knapsack_weight, knapsack_value
        actions = [
            (ki, di, knapsack_value + di.value - ki.value)
            for ki in knapsack
            for di in desk
            if knapsack_weight - ki.weight + di.weight < max_weight
        ]
        actions.extend(
            (None, di, knapsack_value + di.value)
            for di in desk
            if knapsack_weight + di.weight < max_weight
        )
        
        max_v = knapsack_value
        max_ki = None
        max_di = None
        for ki, di, v in actions:
            if v > max_v:
                max_v = v
                max_ki = ki
                max_di = di

        if max_di is not None:
            desk.remove(max_di)
            knapsack.append(max_di)
            knapsack_weight += max_di.weight
            knapsack_value += max_di.value
        if max_ki is not None:
            knapsack.remove(max_ki)
            desk.append(max_ki)
            knapsack_weight -= max_ki.weight
            knapsack_value -= max_ki.value

        return knapsack_value, knapsack_weight
    
    for i in range(max_steps):
        hill_climbing_step()
    
    return knapsack, knapsack_value, knapsack_weight

In [40]:
%time hill_climbing(items, 100, best_first=True)

CPU times: user 6.11 s, sys: 102 ms, total: 6.21 s
Wall time: 6.3 s


([Item(value=16.58239338802688, weight=11.783649467897803),
  Item(value=15.32653594658761, weight=3.8491979265148473),
  Item(value=14.916308429503074, weight=0.1),
  Item(value=14.786799035229176, weight=13.243744547959032),
  Item(value=14.682194621253954, weight=17.686584034340317),
  Item(value=14.538928805587465, weight=5.422369747871384),
  Item(value=14.154797289691974, weight=25.545578075294344),
  Item(value=14.140895463004167, weight=9.990772435986628),
  Item(value=14.078772416893802, weight=5.7452268879308654),
  Item(value=13.994828635477782, weight=0.1),
  Item(value=13.988373309254781, weight=6.219539973153907),
  Item(value=13.784416654399854, weight=0.1),
  Item(value=13.611777618953848, weight=0.1),
  Item(value=13.511080960777235, weight=0.1)],
 202.0981025746416,
 99.98666309694912)

### Simulated annealing, greedy best first, etc

In [41]:
def hill_climbing(items, max_weight, max_steps=100, best_first=False, anneal_rate=0):
    desk = list(items)
    
    knapsack = []
    knapsack_weight = 0
    knapsack_value = 0
    
    def hill_climbing_step():
        nonlocal knapsack_weight, knapsack_value
        actions = [
            (ki, di, knapsack_value + di.value - ki.value)
            for ki in knapsack
            for di in desk
            if knapsack_weight - ki.weight + di.weight < max_weight
        ]
        actions.extend(
            (None, di, knapsack_value + di.value)
            for di in desk
            if knapsack_weight + di.weight < max_weight
        )
        
        if best_first:
            random.shuffle(actions)

        max_v = knapsack_value
        max_ki = None
        max_di = None
        for ki, di, v in actions:
            if v > max_v:
                max_v = v
                max_ki = ki
                max_di = di
                if best_first:
                    break
            else:
                if best_first and (random.random() < anneal_rate):
                    break

        if max_di is not None:
            desk.remove(max_di)
            knapsack.append(max_di)
            knapsack_weight += max_di.weight
            knapsack_value += max_di.value
        if max_ki is not None:
            knapsack.remove(max_ki)
            desk.append(max_ki)
            knapsack_weight -= max_ki.weight
            knapsack_value -= max_ki.value

        return knapsack_value, knapsack_weight
    
    for i in range(max_steps):
        hill_climbing_step()
    
    return knapsack, knapsack_value, knapsack_weight

In [42]:
%time hill_climbing(items, 100, best_first=False)

CPU times: user 6.98 s, sys: 124 ms, total: 7.1 s
Wall time: 7.1 s


([Item(value=16.58239338802688, weight=11.783649467897803),
  Item(value=15.32653594658761, weight=3.8491979265148473),
  Item(value=14.916308429503074, weight=0.1),
  Item(value=14.786799035229176, weight=13.243744547959032),
  Item(value=14.682194621253954, weight=17.686584034340317),
  Item(value=14.538928805587465, weight=5.422369747871384),
  Item(value=14.154797289691974, weight=25.545578075294344),
  Item(value=14.140895463004167, weight=9.990772435986628),
  Item(value=14.078772416893802, weight=5.7452268879308654),
  Item(value=13.994828635477782, weight=0.1),
  Item(value=13.988373309254781, weight=6.219539973153907),
  Item(value=13.784416654399854, weight=0.1),
  Item(value=13.611777618953848, weight=0.1),
  Item(value=13.511080960777235, weight=0.1)],
 202.0981025746416,
 99.98666309694912)

In [43]:
%time hill_climbing(items, 100, best_first=True, anneal_rate=0.1)

CPU times: user 18.4 s, sys: 35.3 ms, total: 18.4 s
Wall time: 18.5 s


([Item(value=12.53700124223899, weight=5.679668056236004),
  Item(value=10.644962731580156, weight=3.2769864653136827),
  Item(value=11.580019796371802, weight=15.10081599450648),
  Item(value=8.973411920872561, weight=9.421369126253383),
  Item(value=9.870583740933945, weight=2.6898886579567964),
  Item(value=10.199390231097864, weight=0.1),
  Item(value=9.05255925174059, weight=0.1),
  Item(value=12.137016509931529, weight=0.1),
  Item(value=9.660594412037277, weight=2.0212194843368447),
  Item(value=10.537242891316756, weight=11.693270831874406),
  Item(value=14.078772416893802, weight=5.7452268879308654),
  Item(value=10.897021231393616, weight=0.1),
  Item(value=11.830514896454954, weight=4.338820383243287),
  Item(value=11.293172497863248, weight=0.8838854034237222),
  Item(value=6.281119706109734, weight=0.1),
  Item(value=9.093291507572369, weight=0.1),
  Item(value=9.201972192148213, weight=2.9888748950285144),
  Item(value=10.556706195514764, weight=0.2830364535335015),
  Ite